# [Chapter 2. Strings and Text](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html)

Almost every useful program involves some kind of processing, whether it is parsing data or generating output.  
This chapter focuses on common problems involving text manipulation, such as pulling apart strings, searching, substitution, lexing, and parsing.  
Many of these tasks can be easily solved using built-in methods of strings.  
However, more complicated operations might require the use of regular expressions or the creation of a full-fledged parser.  
All of those topics will be covered.  
In addition, a few tricky aspects of [working with Unicode](https://en.wikipedia.org/wiki/Unicode) are addressed.  
Let's do this!

## [Splitting Strings on Any of Multiple Delimiters](http://chimera.labs.oreilly.com/books/1230000000393/ch02.html#_splitting_strings_on_any_of_multiple_delimiters)